In [17]:
import numpy as np
import pandas as pd
import math
import re
import requests
from bs4 import BeautifulSoup
from time import sleep

In [26]:
for i in range(2016, 2021):    
    year = str(i)
    url = f"https://www.bls.gov/cps/aa{year}/cpsaat07.htm"
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')


    table_data = []
    for i in soup.find_all('td'):
        if i.get_text() == '':
            pass
        elif re.findall('[a-z]', i.get_text()):
            pass
        else:
            table_data.append(i.get_text())

    clean_data = []
    i = 0
    while i <= len(table_data):
        raw_data = []
        try:
            while len(raw_data) < 8:
                    raw_data.append(table_data[i])
                    i += 1
        except:
            break
        clean_data.append(raw_data)


    outer_index_to_clean = []
    for elements in soup.find_all(class_='sub0'):
            if elements.get_text() == 'Civilian noninstitutional population':
                pass
            else:
                outer_index_to_clean.append(elements.get_text())

    inner_index = []
    for items in soup.find_all(class_=['sub0', 'sub1','sub2', 'sub3', 'sub4']):
        if items.get_text() in outer_index_to_clean:
            pass
        else:
            inner_index.append(items.get_text())

    outer_index = []
    k = 0
    for i in range(0, len(inner_index)):
        try:
            if inner_index[i+1] == 'Civilian noninstitutional population':
                outer_index.append(outer_index_to_clean[k])
                k+=1
            else:
                outer_index.append(outer_index_to_clean[k])
        except:
            outer_index.append(outer_index_to_clean[k])
            
    assert len(outer_index) == len(inner_index)


    column_level1 = np.array(((year + ',') * 8).split(',')[:-1])
    column_level2 = np.array(
        [
        'Less than a high school diploma',
        'High school graduates, no college',
        'Some college or associate degree',
        'Some college or associate degree',
        'Some college or associate degree',
        'Bachelor\'s degree and higher',
        'Bachelor\'s degree and higher',
        'Bachelor\'s degree and higher'
        ]
    )
    column_level3 = np.array(
        [
        'Less than a high school diploma',
        'High school graduates, no college',
        'Total',
        'Some college, no degree',
        'Associate degree',
        'Total',
        'Bachelor\'s degree only',
        'Advanced degree'
        ]
    )

    column_arrays = [column_level1, column_level2, column_level3]
    index_arrays = [outer_index, inner_index]

    df = pd.DataFrame(clean_data, index=index_arrays, columns=column_arrays)

    df.to_pickle(f'./cpsaat-{year}.pkl')
    
    # Be polite to the government servers
    sleep(5)
    
